In [ ]:
# Hard path reference, can be run without installing robot_perception
from robot_perception.src.homography_estimation import normalized_dlt, util
# After installing robot_perception package can run this directly
#from homography_estimation import normalized_dlt, util
import numpy as np
import random
from scipy.ndimage import imread
import matplotlib.pyplot as plt
%matplotlib inline


img1 = imread("boat/img1+points.png")
img2 = imread("boat/img2+points.png")

with open("boat/homography.txt") as f:
    lines = f.readlines()

# Get the text coordinates
lines[0] = lines[0][lines[0].find('[') + 1:lines[0].find(']')]
lines[1] = lines[1][lines[1].find('[') + 1:lines[1].find(']')]

# Split coordinate pairs
lines[0] = lines[0].split(';')
lines[1] = lines[1].split(';')

# Map coordinates to 2D int arrays
x = []
x_tick = []
for i in range(len(lines[0])):
    x.append(list(map(int, lines[0][i].split(','))))
    x_tick.append(list(map(int, lines[1][i].split(','))))

x = np.array(x)
x_tick = np.array(x_tick)

In [ ]:
def gold_standard():
    pass


def get_sampson_corrected_points(x, homography):
    """
    Implement Sampson error correction from equation 4.11, section 4.2.6,
    Hartley's Multiple View Geometry (MVG):
        delta_x = - J.T (JJ.T)^-1 epsilon 
    Refer to exercise (vii), section 4.9.2 in MVG for JJ.T calculation.
    Note: epsilon is identical to the cost function evaluated at X
    :param x: 4-vector (x, y, x', y')
    :param homography: calculated homography to be evaluated
    :return: adjusted (x, y), or x_hat
    """
    epsilon = evaluate_cost_function(x, homography)

    jacobian = None
    e_i = np.zeros(len(x))
    for i in range(len(x)):
        e_i[i] = 1
        j_i = evaluate_cost_function(x + e_i, homography) - epsilon
        if jacobian is None:
            jacobian = j_i
        else:
            jacobian = np.append(jacobian, j_i, axis=1)
        e_i[i] = 0

    jjt = np.dot(jacobian, np.transpose(jacobian))
    lambd = np.linalg.lstsq(jjt, -epsilon)[0]
    delta_x = np.dot(np.transpose(jacobian), lambd).flatten()
    x_hat = x + delta_x

    return x_hat[:2]


def evaluate_cost_function(x, homography):
    """
    Evaluate cost function Ah at X
    :param x: 4-vector (x, y, x', y')
    :param homography: calculated homography matrix
    :return: value of cost function as (2, 1) matrix
    """
    matrix_a = normalized_dlt.construct_A_i_matrix(np.append(x[:2], 1.),
                                                   np.append(x[2:], 1.))
    return np.dot(matrix_a, np.reshape(homography, (9, 1)))

In [ ]:
H = normalized_dlt.get_homography(x, x_tick)
#print(H)
print(np.append(x[3, :], x_tick[3, :]))

print(get_sampson_corrected_points(np.append(x[3, :], x_tick[3, :]), H))